# Deploy the Model

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

![Pipeline](img/generative_ai_pipeline_rlhf_plus.png)

In [2]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=32877658112, available=5233467392, percent=84.1, used=27171418112, free=552476672, active=30701600768, inactive=381702144, buffers=0, cached=5153763328, shared=1794048, slab=493752320)


In [3]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

# Retrieve model endpoint


In [4]:
%store -r pipeline_endpoint_name

In [5]:
try:
    pipeline_endpoint_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run previous notebooks before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [6]:
print(pipeline_endpoint_name)

model-from-registry-ep-1682445517


In [7]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker HTTPS Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_
_Note:  This will take a few minutes.  Please be patient._

In [8]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

WaiterError: Waiter EndpointInService failed: Waiter encountered a terminal failure state: For expression "EndpointStatus" we matched expected path: "Failed"

# _Wait Until the Endpoint ^^ Above ^^ is Deployed_

# Zero Shot Inference

In [9]:
import json
from sagemaker import Predictor

zero_shot_prompt = """Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary:"""
predictor = Predictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
)
response = predictor.predict(zero_shot_prompt,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
)
response_json = json.loads(response.decode('utf-8'))
print(response_json)

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint model-from-registry-ep-1682445517 of account 079002598131 not found.

# Test predictions

In [ ]:
from random import randint
import pandas as pd

In [ ]:
# Load the train and test data into a pandas dataframe
#train_data = pd.read_json("data/dialogsum.train.jsonl", lines=True)
test_data = pd.read_json("data/dialogsum.test.jsonl", lines=True)
#print ("Train data shape: ", train_data.shape)
print ("Test data shape: ", test_data.shape)

In [ ]:
random_dialogue_idx = randint(0, test_data.shape[0])
random_dialogue = test_data["dialogue"][random_dialogue_idx]

output = predictor.predict({"inputs": [random_dialogue], "parameters":{"max_length": 100}})
output_summary = output["outputs"][0]["summary_text"]

print("#####DIALOGUE######\n", random_dialogue)
print("\n#####GENERATED SUMMARY######\n", output_summary)

# Clean Up: Tear Down Endpoint

In [ ]:
sm.delete_endpoint(
    EndpointName=pipeline_endpoint_name
)